In [ ]:
import os
import sys
from pathlib import Path
import logging

from tqdm.notebook import tqdm
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append("..")
from nmaipy.feature_api import FeatureApi
from nmaipy.constants import AREA_CRS, API_CRS
import nmaipy.coverage_utils

SINCE = "2007-01-01"
UNTIL = "2024-01-01"

DATA_DIR = Path("../data")
COVERAGE_CHUNK_DIR = DATA_DIR / "coverage_chunks"
AI_COVERAGE_CHUNK_DIR = DATA_DIR / "ai_coverage_chunks"

POINTS_FILE = DATA_DIR / "2023-10-09-nearmap-backfill-centroids.csv"
ID_COL = "record_id"
API_KEY = os.environ["API_KEY"]

## Load Points
Load a file of points to check for coverage of imagery, 3D and AI.

In [ ]:
df_points = pd.read_csv(POINTS_FILE, index_col=0, parse_dates=["since", "until"])
df_points.since.hist()
df_points.until.hist()
df_points

In [ ]:
df_points["since"] = SINCE
df_points["until"] = UNTIL

In [ ]:
df_points.since.hist()
df_points.until.hist()
df_points

## Test Coverage
For every point, get a full history of all surveys that intersect with it from the coverage API, and whether the survey has AI and/or 3D resources attached. This doesn't tell us what generation of AI data is available - that will require a subsequent run against the AI Feature API coverage endpoint to determine versions (or just pulling the data and ignoring version).

In [ ]:
CHUNK_SIZE = 10000  # For big tests, useful to save chunks of coverage calls in case of failure.

Path(COVERAGE_CHUNK_DIR).mkdir(parents=True, exist_ok=True)

df_coverage = nmaipy.coverage_utils.get_coverage_from_points(
    df_points,
    api_key=API_KEY,
    coverage_type="standard",
    chunk_size=CHUNK_SIZE,
    threads=20,
    coverage_chunk_cache_dir=COVERAGE_CHUNK_DIR,
    id_col=ID_COL,
)

In [ ]:
# The full set of surveys covered by each point, regardless of whether it has AI.
df = df_points.merge(df_coverage, left_index=True, right_index=True, how="right")
df

In [ ]:
# Unique surveys covered by a point.
df_surveys = df.sort_values("aifeatures").drop_duplicates(["survey_id"], keep="last")

# Show count of surveys covered/not covered by AI and 3D.
display(df_surveys.query("tiles == True").aifeatures.value_counts().to_frame())
display(df_surveys["3d"].value_counts().to_frame())

In [ ]:
df_surveys_missing_ai = df_surveys.sort_values("captureDate").query("aifeatures == False & tiles == True")
pd.to_datetime(df_surveys["captureDate"]).hist(bins=30, label="All")
pd.to_datetime(df_surveys_missing_ai["captureDate"]).hist(bins=30, label="Missing AI")
plt.title("Survey Capture Date Distribution (with/without AI)")
plt.legend()
plt.tight_layout()

In [ ]:
# The latest survey for each point within the point's since-until range, regardless of whether it has AI.
df_latest = df.sort_values("captureDate").groupby(ID_COL).last()
df_latest

In [ ]:
# Show what fraction of the latest survey has AI features
df_latest.aifeatures.value_counts() / len(df_latest)

In [ ]:
# Show what fraction of the original points that have at least imagery coverage
f"{len(df_latest) / len(df_points) * 100:.2f}% of the original points that have valid imagery (with or without AI) in the range."

In [ ]:
# The latest survey for each point within the point's since-until range, only if it has AI.
df_ai_latest = df.query("aifeatures == True").sort_values("captureDate").groupby(ID_COL).last()
df_ai_latest

In [ ]:
# Show what fraction have a valid AI result already
display(
    f"{len(df_ai_latest) / len(df_points) * 100:0.1f}% of the originally provided points have valid AI coverage in the since-until range."
)
display(
    f"{len(df_ai_latest) / len(df_latest) * 100:0.1f}% of the points that have at least one valid imagery survey have valid AI coverage in the since-until range."
)

## Which points don't have AI coverage?
It is often useful to explore which points don't have AI coverage yet (but do have imagery coverage). The following section identifies these points, and also has an algorithm to produce a set of surveys which will incrementally increase the AI coverage (a greedy search for valid surveys that match between the largest set of points). This allows us to answer the question "if we ran AI on this list of extra surveys, we could increase AI coverage from x% to y%".

In [ ]:
# Get the subset of points in "df" that have no coverage (i.e. don't fall in "df_ai_latest")
df_no_coverage = df.loc[~df.index.isin(df_ai_latest.index), :]
df_no_coverage

In [ ]:
df_no_coverage_latest = df_no_coverage.sort_values("captureDate").groupby(ID_COL).last()
df_no_coverage_latest

In [ ]:
f"{len(df_no_coverage_latest) / len(df_latest) * 100:0.1f}% of the points that have valid imagery DON'T have valid AI coverage in the since-until range."

In [ ]:
# Iteratively cycle through identifying the most common "survey_id" in df_no_coverage, and then remove all rows that have coverage on that survey_id.

df_surveys_to_run = []
SURVEY_NUMBER_BUDGET = 200

for i in range(SURVEY_NUMBER_BUDGET):
    most_common_survey = df_no_coverage.survey_id.value_counts().index[0]
    capture_date = df_no_coverage.query("survey_id == @most_common_survey").captureDate.iloc[0]
    df_surveys_to_run.append({"survey_id": most_common_survey, "captureDate": capture_date})
    idx_covered_by_most_common_survey = df_no_coverage.query("survey_id == @most_common_survey").index

    df_no_coverage = df_no_coverage.loc[~df_no_coverage.index.isin(idx_covered_by_most_common_survey), :]
df_surveys_to_run = pd.DataFrame(df_surveys_to_run).set_index("survey_id")
df_surveys_to_run

In [ ]:
df_no_coverage_latest = df_no_coverage.sort_values("captureDate").groupby(ID_COL).last()
df_no_coverage_latest

In [ ]:
frac_with_ai_already = len(df_ai_latest.query("aifeatures")) / len(df_latest)
frac_without_ai_after_extra_surveys = len(df_no_coverage_latest) / len(df_latest)
simulated_num_with_coverage = len(df_latest) - len(df_no_coverage_latest)

display(
    f"If an additional {SURVEY_NUMBER_BUDGET} surveys are run with AI, the AI coverage (out of possible imagery surveys) would increase from {frac_with_ai_already*100:0.1f}% to {(1-frac_without_ai_after_extra_surveys)*100:0.1f}%"
)
display(
    f"i.e. {simulated_num_with_coverage:,} points covered in total, out of the original {len(df_points):,} provided ({simulated_num_with_coverage / len(df_points)*100:0.1f}%) after running  {SURVEY_NUMBER_BUDGET} surveys."
)

In [ ]:
df.to_parquet(DATA_DIR / f"{POINTS_FILE.stem}_all.parquet")
df.query("aifeatures").to_parquet(DATA_DIR / f"{POINTS_FILE.stem}_all_with_ai_coverage.parquet")
df_latest.to_parquet(DATA_DIR / f"{POINTS_FILE.stem}_latest_with_ai_coverage.parquet")
df_surveys_to_run.to_csv(DATA_DIR / f"{POINTS_FILE.stem}_surveys_to_run.csv")

# AI System Version
The AI coverage endpoint in the AI Feature API provides more information than the standard coverage API, particularly about systemversion (e.g. `gen5-tranquil_pool-1.0`). In the next section, we take all the results above that have an AI result, and check what version is available.

In [ ]:
df_ai = df.query("aifeatures == True")
df_ai

In [ ]:
df_ai_coverage = None

Path(AI_COVERAGE_CHUNK_DIR).mkdir(parents=True, exist_ok=True)

df_coverage = nmaipy.coverage_utils.get_coverage_from_points(
    df_points,
    api_key=API_KEY,
    coverage_type="ai",
    chunk_size=CHUNK_SIZE,
    threads=20,
    coverage_chunk_cache_dir=AI_COVERAGE_CHUNK_DIR,
    id_col=ID_COL,
)

In [ ]:
# Breakdown of versions in the most recent valid AI result for each point.
df_ai_coverage.sort_values("captureDate").groupby(ID_COL).last().systemVersion.value_counts()